In [1]:
import sys
import os
IMATOOLS_DIR = os.getcwd()+'/..'
sys.path.insert(1, IMATOOLS_DIR)


In [2]:
import imatools.vtktools as vtktools
import imatools.ioutils as iou
import platform as pltf

# Locations of hard drive based on platform
dirdic={'MacOS'   : '/Volumes/sandisk', 
        'Linux'   : '/media/jsl19/sandisk' , 
        'Windows' : 'D:/'}

p2f = iou.fullfile(dirdic[pltf.system()], '09-dnav_vs_inav/umc/local')
N = os.listdir(p2f)
xnav = lambda x : 'LGE_' + x + 'NAV'


In [3]:
import vtk 
n = 1  # 0..22 (23 cases)
inav = iou.fullfile(p2f, str(N[n]), xnav('i'), 'OUTPUT')
dnav = iou.fullfile(p2f, str(N[n]), xnav('d'), 'OUTPUT')

dpath = iou.searchFileByType(dnav, 'IIR_MaxScar', 'vtk')[0]
ipath = iou.searchFileByType(inav, 'IIR_MaxScar', 'vtk')[0]

msh_d = vtktools.readVtk(dpath) # source
msh_i = vtktools.readVtk(ipath) # target

omsh = vtk.vtkPolyData()
omsh.DeepCopy(msh_d)

target_pl = vtk.vtkPointLocator()
target_pl.SetDataSet(msh_i)
target_pl.AutomaticOn()
target_pl.BuildLocator()

target_scalar = msh_i.GetCellData().GetScalars()
o_scalar = vtk.vtkFloatArray()
o_scalar.SetNumberOfComponents(1)

default_value=0
for ix in range(msh_d.GetNumberOfPoints()) :
    pt = msh_d.GetPoint(ix)
    id_on_target = target_pl.FindClosestPoint(pt)

    mapped_val = target_scalar.GetTuple1(id_on_target) if (id_on_target > 0) else default_value
    o_scalar.InsertNextTuple1(mapped_val)

omsh.GetPointData().SetScalars(o_scalar)
        

1

In [4]:
import numpy as np 

cog = vtktools.getCentreOfGravity(msh_d)

print(cog)

TypeError: Cannot interpret '3' as a data type